210809 <전처리>

1. 중복값 제거 ok

2. 장르 : 국적과 있는 데이터 처리 ok
 매출액 : 음수 처리 ok
누적매출액 : 음수 처리 ok

3. 대표국적
- 결측 & 기타 -> 기타  ok
- 상위 10개 국가  ok
- 나머지 국가는 기타로 처리  ok

4. 배급사 : 1개
- 결측 -> 0  ok
- 개봉일 결측 -> 0  ok
- 3년치 데이터 계산

5. 등급 ok
- [국내] 문자 제거 ok
- 결측 & 기타 -> 등급없음 ok
- 등급없음 포함 5개의 카테고리로 나눔 ok

6. 장르 o
- 가장 앞의 장르만 남김 ok
- 결측 -> 장르없음 ok
- 21개의 카테고리로 나눔 ok
- 이상한 값들 -> 장르없음 ok

7. 감독 : 1명
- 결측 -> 0  ok
- 개봉일 결측 -> 0  ok
- 3년치 데이터 계산

8. 배우 : 최대 3명 
- 결측 -> 0  ok
- 개봉일 결측 -> 0  ok
- 3년치 데이터 계산

9. 매출액 관객수 ok
음수 -> 양수 ok

In [245]:
import requests
from bs4 import BeautifulSoup
import time
import xlsxwriter as xlsxwriter
import csv
import math
import re
from datetime import datetime
import numpy as np
import pandas as pd

## preprocessing1

In [246]:
pd.set_option('display.max_rows',1000)

In [247]:
df_raw = pd.read_excel("crawl_total.xlsx", index_col = False, thousands= ',')
# df_genne = pd.read_csv("genre.csv", index_col = False)
# df_raw = pd.concat([df_raw, df_genne])
df = df_raw
df

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,태극기 휘날리며,2004-02-05,15687180500,15687180500,2544911,2544911,115,22375,한국,"(주)다자인소프트,(주)쇼박스",15세관람가,"전쟁,드라마",강제규,"장동건,원빈,이은주,공형진,조원희,이장욱,이영란,정희준,박유신,김효열,김효준,최재혁..."
1,트로이,2004-05-21,12777346000,12777346000,2001293,2001293,177,21245,미국,워너브러더스 코리아(주),15세관람가,사극,볼프강 페터젠,"다이앤 크루거,올랜도 블룸,에릭 바나"
2,내 머리 속의 지우개,2004-11-05,11919939500,11919939500,1885827,1885827,206,29532,한국,CJ ENM,12세관람가,멜로/로맨스,이재한,"정우성,손예진,데이비드 맥기니스,김부선,정광재,최기호,박경진,장재용,전은희,이영필,..."
3,귀신이 산다,2004-09-17,11898748500,11898748500,1875936,1875936,174,27936,한국,(주)시네마서비스,12세관람가,"코미디,공포(호러),판타지",김상진,"김응수,장서희,차승원,김서현,최기웅,유대현,김성미,태인호,임상란,장승현,김정우,김양..."
4,투모로우,2004-06-03,11658584500,11658584500,1830767,1830767,164,22881,미국,이십세기폭스필름코퍼레이션,12세관람가,SF,롤랜드 에머리히,"데니스 퀘이드,제이크 질렌할,이안 홈"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27228,"그대, 고맙소 : 김호중 생애 첫 팬미팅 무비",2020-09-29,-1164993296,-1164993296,102008,102008,106,4771,한국,씨제이포디플렉스 주식회사,전체관람가,공연,없음,김호중
27229,"[통합]회고전, 기획전 등",NaN,985000,501984300,146,104955,1,3,NaN,NaN,NaN,장르없음,없음,NaN
27230,2006 [통합]영화제/기획전/회고전 등,NaN,1938000,583092200,226,106619,1,4,NaN,NaN,NaN,장르없음,없음,NaN
27231,프리즌 이스케이프,2020-05-06,1848297600,1848297600,217056,217056,547,29305,영국,(주)이놀미디어,12세이상관람가,어드벤처,없음,"다니엘 래드클리프,이안 하트"


In [248]:
df = df.loc[:, "영화명": "배우"]
df

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,태극기 휘날리며,2004-02-05,15687180500,15687180500,2544911,2544911,115,22375,한국,"(주)다자인소프트,(주)쇼박스",15세관람가,"전쟁,드라마",강제규,"장동건,원빈,이은주,공형진,조원희,이장욱,이영란,정희준,박유신,김효열,김효준,최재혁..."
1,트로이,2004-05-21,12777346000,12777346000,2001293,2001293,177,21245,미국,워너브러더스 코리아(주),15세관람가,사극,볼프강 페터젠,"다이앤 크루거,올랜도 블룸,에릭 바나"
2,내 머리 속의 지우개,2004-11-05,11919939500,11919939500,1885827,1885827,206,29532,한국,CJ ENM,12세관람가,멜로/로맨스,이재한,"정우성,손예진,데이비드 맥기니스,김부선,정광재,최기호,박경진,장재용,전은희,이영필,..."
3,귀신이 산다,2004-09-17,11898748500,11898748500,1875936,1875936,174,27936,한국,(주)시네마서비스,12세관람가,"코미디,공포(호러),판타지",김상진,"김응수,장서희,차승원,김서현,최기웅,유대현,김성미,태인호,임상란,장승현,김정우,김양..."
4,투모로우,2004-06-03,11658584500,11658584500,1830767,1830767,164,22881,미국,이십세기폭스필름코퍼레이션,12세관람가,SF,롤랜드 에머리히,"데니스 퀘이드,제이크 질렌할,이안 홈"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27228,"그대, 고맙소 : 김호중 생애 첫 팬미팅 무비",2020-09-29,-1164993296,-1164993296,102008,102008,106,4771,한국,씨제이포디플렉스 주식회사,전체관람가,공연,없음,김호중
27229,"[통합]회고전, 기획전 등",NaN,985000,501984300,146,104955,1,3,NaN,NaN,NaN,장르없음,없음,NaN
27230,2006 [통합]영화제/기획전/회고전 등,NaN,1938000,583092200,226,106619,1,4,NaN,NaN,NaN,장르없음,없음,NaN
27231,프리즌 이스케이프,2020-05-06,1848297600,1848297600,217056,217056,547,29305,영국,(주)이놀미디어,12세이상관람가,어드벤처,없음,"다니엘 래드클리프,이안 하트"


### 2. 중복값 제거

In [249]:
df = df.sort_values(["감독","누적관객수","누적매출액"], ascending=True)

In [250]:
df.drop_duplicates(['영화명',"감독"], keep = 'last',inplace=True)

In [251]:
df["영화명"].value_counts()

로미오와 줄리엣                                           6
카르멘                                                6
토스카                                                6
욕망                                                 5
첫사랑                                                4
                                                  ..
가슴 사무라이-이단의 역습                                     1
36시간                                               1
스페셜 시큐리티 : 라스트 미션                                  1
미라클 문 페스티벌 츠키우타. 버추얼 라이브 2019 사계 딜레이 뷰잉 ~여름 버전~    1
꽃과 뱀: 더 제로                                         1
Name: 영화명, Length: 20749, dtype: int64

### 3. 장르 전처리

In [252]:
df["장르"].value_counts()

드라마                              4336
멜로/로맨스                           2036
다큐멘터리                            2001
애니메이션                            1810
액션                               1452
장르없음                             1282
코미디                              1025
에로                                943
성인물(에로)                           759
공포(호러)                            549
스릴러                               462
범죄                                366
SF                                244
미스터리                              161
판타지                               132
공포                                132
공연                                129
전쟁                                126
가족                                 93
어드벤처                               82
모험                                 79
뮤지컬                                61
드라마,멜로/로맨스                         57
멜로/로맨스,코미디                         51
코미디,드라마                            47
서부                                 45
공연실황        

In [253]:
## nan값 확인 후 처리
df["장르"].isnull().sum() #결측값 1843

1843

In [254]:
df.tail()

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
23597,댄싱 인 자파,NaN,88000,88000,15,15,1,2,미국,NaN,NaN,다큐멘터리,힐라 메달리아,NaN
26526,위기의 30대 여자들,NaN,946400,946400,136,136,1,2,이스라엘,NaN,NaN,다큐멘터리,힐라 메달리아,NaN
23598,16세의 사운드트랙,2020-07-02,3543400,3543400,413,413,19,150,영국,(주)마노엔터테인먼트,12세이상관람가,코미디,힐러리 셰익스피어,"제임스 캘로웨이,스칼렛 마셜"
2786,나의 말을 지켜봐: 어느 귀 먼 연예인들의 이야기,NaN,293500,293500,114,114,1,2,미국,NaN,NaN,다큐멘터리,힐러리 스칼,NaN
23599,분노의 건샷,2016-12-14,4000,4000,1,1,1,1,미국,(주) 케이알씨지,[국내] 15세 관람가,범죄,힐러리안 뱅크스,"제임스 와일더,조아큄 드 알메이다,레베카 메이 파머"


In [255]:
df.reset_index(inplace=True)

In [256]:
df = df.loc[:, "영화명": "배우"]
df

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,"416 프로젝트 ""공동의 기억: 트라우마""",NaN,2192500,2192500,437,437,3,17,한국,NaN,NaN,다큐멘터리,416연대 미디어위원회,NaN
1,미씨카: 더 네크로맨서,2017-11-02,300000,300000,60,60,1,1,미국,(주)제이에이와이이엔터테인먼트,15세이상관람가,판타지,A. 토드 스미스,"애덤 존슨,로버트 제인,케빈 소르보"
2,월드 워 핵전쟁의 서막,2019-03-07,26000,26000,17,17,5,17,뉴질랜드,(주)풍경소리,[국내] 15세 관람가,액션,A.K. 스트롬,"모건 브래들리,그레이엄 빈센트,캠벨 루셀"
3,울프라이징,2016-11-17,300000,300000,60,60,1,1,미국,주식회사 루믹스미디어,15세이상관람가,공포(호러),BC 퍼트니,빌 오버스트 주니어
4,제3의앵글 : 형형색색대만전,NaN,1830000,1830000,305,305,1,2,대만,NaN,NaN,NaN,BITO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21320,댄싱 인 자파,NaN,88000,88000,15,15,1,2,미국,NaN,NaN,다큐멘터리,힐라 메달리아,NaN
21321,위기의 30대 여자들,NaN,946400,946400,136,136,1,2,이스라엘,NaN,NaN,다큐멘터리,힐라 메달리아,NaN
21322,16세의 사운드트랙,2020-07-02,3543400,3543400,413,413,19,150,영국,(주)마노엔터테인먼트,12세이상관람가,코미디,힐러리 셰익스피어,"제임스 캘로웨이,스칼렛 마셜"
21323,나의 말을 지켜봐: 어느 귀 먼 연예인들의 이야기,NaN,293500,293500,114,114,1,2,미국,NaN,NaN,다큐멘터리,힐러리 스칼,NaN


In [257]:
df

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,"416 프로젝트 ""공동의 기억: 트라우마""",NaN,2192500,2192500,437,437,3,17,한국,NaN,NaN,다큐멘터리,416연대 미디어위원회,NaN
1,미씨카: 더 네크로맨서,2017-11-02,300000,300000,60,60,1,1,미국,(주)제이에이와이이엔터테인먼트,15세이상관람가,판타지,A. 토드 스미스,"애덤 존슨,로버트 제인,케빈 소르보"
2,월드 워 핵전쟁의 서막,2019-03-07,26000,26000,17,17,5,17,뉴질랜드,(주)풍경소리,[국내] 15세 관람가,액션,A.K. 스트롬,"모건 브래들리,그레이엄 빈센트,캠벨 루셀"
3,울프라이징,2016-11-17,300000,300000,60,60,1,1,미국,주식회사 루믹스미디어,15세이상관람가,공포(호러),BC 퍼트니,빌 오버스트 주니어
4,제3의앵글 : 형형색색대만전,NaN,1830000,1830000,305,305,1,2,대만,NaN,NaN,NaN,BITO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21320,댄싱 인 자파,NaN,88000,88000,15,15,1,2,미국,NaN,NaN,다큐멘터리,힐라 메달리아,NaN
21321,위기의 30대 여자들,NaN,946400,946400,136,136,1,2,이스라엘,NaN,NaN,다큐멘터리,힐라 메달리아,NaN
21322,16세의 사운드트랙,2020-07-02,3543400,3543400,413,413,19,150,영국,(주)마노엔터테인먼트,12세이상관람가,코미디,힐러리 셰익스피어,"제임스 캘로웨이,스칼렛 마셜"
21323,나의 말을 지켜봐: 어느 귀 먼 연예인들의 이야기,NaN,293500,293500,114,114,1,2,미국,NaN,NaN,다큐멘터리,힐러리 스칼,NaN


In [258]:
for i in range(len(df["장르"])):
    if type(df["장르"][i]) == float:
        df["장르"][i] = "장르없음"
    else:
        df["장르"][i] = df["장르"][i].split(",")[0]

<ipython-input-258-ce42fc033b6c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["장르"][i] = df["장르"][i].split(",")[0]
<ipython-input-258-ce42fc033b6c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["장르"][i] = "장르없음"


In [259]:
# 카테고리 만들기

df.replace("성인물(에로)", "에로", inplace = True)
df.replace("서부극(웨스턴)", "서부", inplace = True)
df.replace("공연실황", "공연", inplace = True)
df.replace("서스펜스", "스릴러", inplace = True)
df.replace("공포(호러)", "공포", inplace = True)
df.replace("멜로/로맨스", "멜로로맨스", inplace = True)
df.replace("실험", "다큐멘터리", inplace = True)

In [260]:
#기타장르 제거
guitar_idx = df[df["장르"]=="기타"].index
df.drop(guitar_idx, inplace=True)

In [261]:
df["장르"].value_counts()

드라마      4552
장르없음     3125
멜로로맨스    2167
다큐멘터리    2014
애니메이션    1899
에로       1711
액션       1586
코미디      1173
공포        756
스릴러       510
범죄        410
SF        295
미스터리      180
공연        172
판타지       149
전쟁        132
가족        117
어드벤처      102
모험         79
뮤지컬        68
서부         56
사극         53
느와르         9
무협          8
서사          2
Name: 장르, dtype: int64

### 4. 매출액  / 누적매출액 : 음수 -> 양수

In [262]:
values = []

for value in df['매출액'].values:
    value = abs(value)
    values.append(value)

In [263]:
stacked_values = []

for value in df['누적매출액'].values:
    value = abs(value)
    stacked_values.append(value)

In [264]:
# df['매출액'] = values
# df['누적매출액'] = stacked_values

In [265]:
df[df['누적매출액'] < 0]
df[df['매출액'] < 0]

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
14,트리플 엑스 리턴즈,2017-02-08,-327632201,-327632201,475309,475309,590,20692,미국,롯데쇼핑㈜롯데엔터테인먼트,15세이상관람가,액션,D.J. 카루소,"빈 디젤,사무엘 L. 잭슨,루비 로즈,니나 도브레브,토니 콜렛,디피카 파드콘,토니 자"
27,맨 인 블랙: 인터내셔널,2019-06-12,-1125725022,-1125725022,851917,851917,1151,40520,미국,롯데컬처웍스(주)롯데엔터테인먼트,12세이상관람가,액션,F. 게리 그레이,"크리스 헴스워스,테사 톰슨,리암 니슨,엠마 톰슨"
47,슈퍼 에이트,2011-06-16,-671828796,-671828796,472157,472157,411,21884,미국,(주)씨제이이엔엠,12세이상관람가,SF,J.J. 에이브럼스,"조엘 코트니,엘르 패닝"
51,스타워즈: 깨어난 포스,2015-12-17,-1710974741,-1710974741,3273879,3273879,940,69226,미국,월트디즈니컴퍼니코리아(주),12세이상관람가,액션,J.J. 에이브럼스,"해리슨 포드,마크 해밀,캐리 피셔,사이먼 페그,오스카 아이삭,막스 본 시도우,도널 글리슨"
69,글래스,2019-01-17,-17514314,-17514314,466475,466475,837,24194,미국,월트디즈니컴퍼니코리아 유한책임회사,15세이상관람가,드라마,M. 나이트 샤말란,"제임스 맥어보이,브루스 윌리스,사무엘 L. 잭슨,안야 테일러 조이,사라 폴슨"
70,애프터 어스,2013-05-30,-463111129,-463111129,537920,537920,484,19867,미국,소니픽쳐스릴리징월트디즈니스튜디오스코리아(주),12세이상관람가,SF,M. 나이트 샤말란,"윌 스미스,제이든 스미스"
124,로그 원: 스타워즈 스토리,2016-12-28,-374508796,458061002,422822,1019309,810,17226,미국,월트디즈니컴퍼니코리아 유한책임회사,12세이상관람가,액션,가렛 에드워즈,"펠리시티 존스,디에고 루나,매즈 미켈슨,리즈 아메드,포레스트 휘테커,견자단,알란 터..."
198,킹 아서: 제왕의 검,2017-05-17,-921288641,-921288641,415714,415714,588,20315,영국,워너브러더스 코리아(주),12세이상관람가,판타지,가이 리치,"찰리 허냄,주드 로,에릭 바나,에이단 길렌,디몬 하운수,아스트리드 베흐제-프리스베,..."
200,셜록홈즈 : 그림자 게임,2011-12-21,-508955684,-508955684,2239156,2239156,370,45652,미국,워너브러더스 코리아(주),15세이상관람가,액션,가이 리치,"로버트 다우니 주니어,주드 로"
320,"고산자, 대동여지도",2016-09-07,-788927334,-788927334,974262,974262,765,38048,한국,(주)씨제이이엔엠,전체관람가,사극,강우석,"김율호,차승원,유준상,김인권,남지현,신동미,남경읍,태인호,공형진,김종수,이형길,임경..."


### 5. 등급 전처리

In [266]:
# 등급에서 "[국내] " 없애기
df['등급'].value_counts()

#df.replace("성인물(에로)", "에로", inplace = True)
df.replace([" 청소년 관람불가 ","청소년 관람불가 ","청소년 관람불가","[국내] 청소년 관람불가","고등학생이상관람가,청소년관람불가","청소년관람불가,12세관람가","국민학생관람불가,청소년관람불가","청소년관람불가,고등학생이상관람가","18세관람가,청소년관람불가","18세 미만인 자는 관람할 수 없는 등급","미성년자관람불가",'18세관람가', "연소자관람불가","연소자관람불가,청소년관람불가"], '청소년관람불가',inplace=True)
df.replace([" 15세 관람가 ","15세 관람가 ","[국내] 15세 관람가"," 15세 관람가","15세관람가","고등학생이상관람가","고등학생이상관람가,15세이상관람가","15세 미만인 자는 관람할 수 없는 등급","15세관람가,15세이상관람가","15세이상관람가,중학생이상관람가","15세이상관람가,미성년자관람불가","청소년관람불가,15세이상관람가","연소자관람불가,15세이상관람가","국민학생관람불가,15세이상관람가","15세이상관람가,전체관람가","15세이상관람가,18세 미만인 자는 관람할 수 없는 등급","연소자관람가,15세이상관람가","18세관람가,15세이상관람가","15세 미만인 자는 관람할 수 없는 등급 ,15세이상관람가"], '15세이상관람가',inplace=True)
df.replace([" 12세 관람가 ","12세 관람가 "," 12세 관람가","[국내] 12세 관람가","12세관람가","중학생이상관람가","국민학생관람불가","12세이상관람가,12세관람가","12세 미만인 자는 관람할 수 없는 등급","12세이상관람가,중학생이상관람가","12세이상관람가,국민학생관람불가","12세이상관람가,전체관람가","12세이상관람가,15세 미만인 자는 관람할 수 없는 등급","12세이상관람가,15세이상관람가","12세이상관람가,고등학생이상관람가","12세이상관람가,국민학생관람불가,15세이상관람가","12세이상관람가,연소자관람가","15세관람가,12세이상관람가"],"12세이상관람가", inplace=True)
df.replace(["전체관람가,중학생이상관람가"," 전체 관람가 ","전체 관람가 ","[국내] 전체 관람가","연소자관람가","모든 관람객이 관람할 수 있는 등급","연소자관람가,전체관람가","청소년관람불가,전체관람가","미성년자관람가","모든 관람객이 관람할 수 있는 등급,전체관람가","12세이상관람가,연소자관람가,전체관람가"],"전체관람가",inplace=True)

In [267]:
df["등급"].value_counts()

청소년관람불가     6316
15세이상관람가    3887
12세이상관람가    2173
전체관람가       1890
Name: 등급, dtype: int64

### 6. 감독, 배우, 배급사

In [268]:
# 감독, 배우, 배급사 결측값 0으로 처리

df['배우'] = df['배우'].fillna(0)
df['감독'] = df['감독'].fillna(0)
df['배급사'] = df['배급사'].fillna(0)
df['등급'] = df['등급'].fillna("등급없음")

In [269]:
df

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,"416 프로젝트 ""공동의 기억: 트라우마""",NaN,2192500,2192500,437,437,3,17,한국,0,등급없음,다큐멘터리,416연대 미디어위원회,0
1,미씨카: 더 네크로맨서,2017-11-02,300000,300000,60,60,1,1,미국,(주)제이에이와이이엔터테인먼트,15세이상관람가,판타지,A. 토드 스미스,"애덤 존슨,로버트 제인,케빈 소르보"
2,월드 워 핵전쟁의 서막,2019-03-07,26000,26000,17,17,5,17,뉴질랜드,(주)풍경소리,15세이상관람가,액션,A.K. 스트롬,"모건 브래들리,그레이엄 빈센트,캠벨 루셀"
3,울프라이징,2016-11-17,300000,300000,60,60,1,1,미국,주식회사 루믹스미디어,15세이상관람가,공포,BC 퍼트니,빌 오버스트 주니어
4,제3의앵글 : 형형색색대만전,NaN,1830000,1830000,305,305,1,2,대만,0,등급없음,장르없음,BITO,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21320,댄싱 인 자파,NaN,88000,88000,15,15,1,2,미국,0,등급없음,다큐멘터리,힐라 메달리아,0
21321,위기의 30대 여자들,NaN,946400,946400,136,136,1,2,이스라엘,0,등급없음,다큐멘터리,힐라 메달리아,0
21322,16세의 사운드트랙,2020-07-02,3543400,3543400,413,413,19,150,영국,(주)마노엔터테인먼트,12세이상관람가,코미디,힐러리 셰익스피어,"제임스 캘로웨이,스칼렛 마셜"
21323,나의 말을 지켜봐: 어느 귀 먼 연예인들의 이야기,NaN,293500,293500,114,114,1,2,미국,0,등급없음,다큐멘터리,힐러리 스칼,0


### 7. 대표국적

In [270]:
nation = ["미국", "한국", "일본", "프랑스", "영국", "독일", "중국", "이탈리아", "캐나다", "스페인"]

for i in range(len(df)):
    if df.iloc[i]["대표국적"] not in nation:
        df["대표국적"].replace(df.iloc[i]["대표국적"],"기타",inplace=True)

In [271]:
# nation = ["미국", "한국", "일본", "프랑스", "영국", "독일", "중국", "이탈리아", "캐나다", "스페인"]

# for i in df["대표국적"]:
#     if df.iloc[i] not in nation:
#         df["대표국적"].replace(df.iloc[i]["대표국적"],"기타",inplace=True)

In [272]:
df["대표국적"].value_counts()

미국      4997
한국      4938
일본      4194
기타      3396
프랑스     1119
영국       839
독일       452
중국       437
이탈리아     390
캐나다      305
스페인      258
Name: 대표국적, dtype: int64

### 중간 저장

In [273]:
df.to_csv("중간저장.csv" ,encoding = 'utf-8-sig')

In [274]:
df = pd.read_csv("중간저장.csv", index_col = False, thousands = ',')
df = df.loc[:, "영화명" : "배우"]
df

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,"416 프로젝트 ""공동의 기억: 트라우마""",NaN,2192500,2192500,437,437,3,17,한국,0,등급없음,다큐멘터리,416연대 미디어위원회,0
1,미씨카: 더 네크로맨서,2017-11-02,300000,300000,60,60,1,1,미국,(주)제이에이와이이엔터테인먼트,15세이상관람가,판타지,A. 토드 스미스,"애덤 존슨,로버트 제인,케빈 소르보"
2,월드 워 핵전쟁의 서막,2019-03-07,26000,26000,17,17,5,17,기타,(주)풍경소리,15세이상관람가,액션,A.K. 스트롬,"모건 브래들리,그레이엄 빈센트,캠벨 루셀"
3,울프라이징,2016-11-17,300000,300000,60,60,1,1,미국,주식회사 루믹스미디어,15세이상관람가,공포,BC 퍼트니,빌 오버스트 주니어
4,제3의앵글 : 형형색색대만전,NaN,1830000,1830000,305,305,1,2,기타,0,등급없음,장르없음,BITO,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21320,댄싱 인 자파,NaN,88000,88000,15,15,1,2,미국,0,등급없음,다큐멘터리,힐라 메달리아,0
21321,위기의 30대 여자들,NaN,946400,946400,136,136,1,2,기타,0,등급없음,다큐멘터리,힐라 메달리아,0
21322,16세의 사운드트랙,2020-07-02,3543400,3543400,413,413,19,150,영국,(주)마노엔터테인먼트,12세이상관람가,코미디,힐러리 셰익스피어,"제임스 캘로웨이,스칼렛 마셜"
21323,나의 말을 지켜봐: 어느 귀 먼 연예인들의 이야기,NaN,293500,293500,114,114,1,2,미국,0,등급없음,다큐멘터리,힐러리 스칼,0


## preprocessing2

+ 배급사 / 감독 / 배우 전처리

### 8. 배급사

In [275]:
# 배급사 1명만 남기기

for i in range(len(df["배급사"])):
    if type(df["배급사"][i]) == float:
        df["배급사"][i] = 0
    else:
        df["배급사"][i] = df["배급사"][i].split(",")[0]

<ipython-input-275-9646cd81362d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배급사"][i] = df["배급사"][i].split(",")[0]


In [276]:
# 배급사 공백 제거

for i in range(len(df["배급사"])):
    if type(df["배급사"][i]) == float:
        df["배급사"][i] = 0
    else:
        List = df["배급사"][i].split(" ")
        df["배급사"][i] = "".join(List)

<ipython-input-276-f92b1096c836>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배급사"][i] = "".join(List)


In [277]:
df["배급사"].replace(["씨제이씨지브이(CJCGV)(주)", "CJ엔터테인먼트", "CJE&M(주)투니버스", "CJE&MPictures", "씨제이포디플렉스주식회사", "(주)씨제이이엔엠"], "CJENM", inplace = True)

df["배급사"].replace(["(주)시네마서비스", "시네마서비스", "필라멘트픽쳐스", "투니버스(주)온미디어", "CGV아트하우스", "CGVICECON"], "CJENM", inplace = True)

df["배급사"].replace(["롯데컬처웍스(주)롯데엔터테인먼트", "롯데쇼핑㈜롯데시네마", "롯데엔터테인먼트"], "롯데쇼핑㈜롯데엔터테인먼트", inplace = True)

df["배급사"].replace(["소니픽쳐스릴리징월트디즈니스튜디오스코리아(주)", "소니픽쳐스릴리징코리아", "소니픽쳐스엔터테인먼트코리아주식회사극장배급지점"], "한국소니픽쳐스릴리징브에나비스타영화㈜", inplace = True)

df["배급사"].replace(["㈜메가박스", "메가박스", "메가박스중앙(주)", "메가박스중앙(주)MEGABOX"], "메가박스중앙(주)플러스엠", inplace = True)

df["배급사"].replace(["가을엔터테인먼트"], "(주)영화사가을", inplace = True)

df["배급사"].replace(["(주)예지림엔터테인먼트"], "예지림엔터테인먼트", inplace = True)

df["배급사"].replace(["디스테이션"], "(주)디스테이션 ", inplace = True)

df["배급사"].replace(["(주)씨지브이신도림"], "(주)프라임엔터테인먼트", inplace = True)

In [278]:
df["배급사"].replace(["NEW","(주)NEW","(주)콘텐츠판다", "콘텐츠판다"],"(주)넥스트엔터테인먼트월드(NEW)",inplace=True)

df["배급사"].replace(["이십세기폭스필름코퍼레이션", "20세기폭스코리아"],"이십세기폭스코아(주)",inplace=True)

df["배급사"].replace(["시네마서비스", "플레너스(주)시네마서비스"],"(주)시네마서비스",inplace=True)

df["배급사"].replace(["월트디즈니컴퍼니코리아(주)", "월트디즈니코리아㈜", "월트디즈니컴퍼니코리아"],"월트디즈니컴퍼니코리아유한책임회사",inplace=True)

df["배급사"].replace(["(주)코리아스크린"],"㈜코리아스크린",inplace=True)

df["배급사"].replace(["(주)풍경소리"],"주식회사풍경소리",inplace=True)

df["배급사"].replace(["시네마리퍼블릭"],"(주)시네마리퍼블릭",inplace=True)

df["배급사"].replace(["레드아이스"],"주식회사레드아이스",inplace=True)

In [279]:
df["배급사"].replace("유니버설픽쳐스인터내셔널코리아",inplace=True)
df["배급사"].replace(["(주)마운틴픽쳐스","마운틴픽처스","마운틴픽쳐스"],"주식회사마운틴픽쳐스",inplace=True)
df["배급사"].replace("팝엔터테인먼트","(주)팝엔터테인먼트",inplace=True)
df["배급사"].replace("디스테이션","(주)디스테이션 ",inplace=True)
df["배급사"].replace("드림팩트엔터테인먼트","(주)드림팩트엔터테인먼트",inplace=True)
df["배급사"].replace("나우콘텐츠","(주)나우콘텐츠",inplace=True)
df["배급사"].replace(["㈜싸이더스픽쳐스","싸이더스"],"(주)싸이더스",inplace=True)
df["배급사"].replace("스폰지","(주)스폰지이엔티",inplace=True)
df["배급사"].replace(["이수C&E","이수문화컨텐츠투자조합"],"(주)이수C&E",inplace=True)
df["배급사"].replace("얼리버드","(주)얼리버드픽쳐스",inplace=True)
df["배급사"].replace("(주)안다미로","(주)영화사안다미로",inplace=True)
df["배급사"].replace("(주)루믹스미디어","주식회사루믹스미디어",inplace=True)
df["배급사"].replace(["워너브러더스사㈜","워너브러더스픽쳐스"],"워너브러더스코리아(주)",inplace=True)
df["배급사"].replace("인디스토리","㈜인디스토리",inplace=True)
df["배급사"].replace("인디플러그","(주)인디플러그",inplace=True)
df["배급사"].replace(["대원디지털엔터테인먼트","대원씨앤에이홀딩스"],"대원미디어(주)",inplace=True)
df["배급사"].replace("소나무미디어","(주)소나무픽쳐스",inplace=True)
df["배급사"].replace("씨네그루(주)키다리이엔티","(주)키다리이엔티",inplace=True)
df["배급사"].replace("(주)브릿지웍스","브릿지웍스엔터테인먼트(주)",inplace=True)

In [280]:
df.replace('(주)루믹스미디어', '주식회사루믹스미디어', inplace=True)
df.replace('(주)안다미로', '(주)영화사안다미로', inplace=True)
df.replace('드림팩트엔터테인먼트', '(유)조이앤컨텐츠그룹', inplace=True)
df.replace('(주)드림팩트엔터테인먼트', '(유)조이앤컨텐츠그룹', inplace=True)
df.replace('디스테이션', '(주)디스테이션', inplace=True)
df.replace('플레너스(주)시네마서비스', '(주)시네마서비스', inplace=True)
df.replace('시네마서비스', '(주)시네마서비스', inplace=True)
df.replace('조이앤시네마', '(유)조이앤컨텐츠그룹', inplace=True)
df.replace('(유)조이앤시네마', '(유)조이앤컨텐츠그룹', inplace=True)
df.replace('(주)케이알씨지', '(유)조이앤컨텐츠그룹', inplace=True)

In [281]:
count = df["배급사"].value_counts()

movies = []
for key, value in count.items():
    if value == 1:
        movies.append(key)

In [282]:
movies

['(주)콘텐츠패밀리',
 '드림로드',
 '(주)코리아준',
 '엔넷커뮤니케이션(주)',
 '(주)프리머스시네마',
 '제이에스필름',
 '롤러코스터',
 '아이비젼엔터테인먼트(주)',
 '㈜기억속의매미',
 '토에이',
 '컴퍼니엘론(주)',
 '푸른영상',
 '독립프로덕션빨간눈사람',
 '(주)테라맥스엔터테인먼트',
 '씨네마캣픽쳐스',
 '강우석프로덕션',
 '서울동화',
 '페어팍스인터내셔날',
 '시나브로엔터테인먼트',
 '(주)노리픽쳐스',
 '애니메이션배급',
 '노가리필름',
 '고몽',
 '미디어존',
 '우성엔터테인먼트',
 '(주)콘텐츠게이트',
 '모두를위한극장공정영화협동조합',
 'SBS콘텐츠허브',
 '미라클필름',
 '(주)엠엔에프씨',
 '(주)온비즈넷',
 '에이아이씨코리아(AICKOREA)',
 '(주)H.F.G엔터테인먼트',
 '(주)글뫼',
 '백필름즈',
 '(주)노시스컴퍼니',
 '무비센트',
 '미래로엔터테인먼트',
 '펜타브리드',
 '52DAYSFILM',
 '(유)씨에이치엔터테인먼트',
 '조이월드앤다이버시티',
 '마오필름(주)',
 '레인주니어픽쳐스',
 '웃기씨네',
 '(주)룬컴퍼니',
 '(주)영화사선',
 '한코리아',
 '굿타임엔터테인먼트',
 '유한회사헤이데이웍스',
 '(주)AVA엔터테인먼트',
 '(주)스튜디오210',
 '스마트스터디(주)',
 '㈜스토리셋',
 '토호영상사업부',
 '에스케이플래닛(주)',
 '(주)시네마뉴원',
 '월드영화사',
 '머스트씨무비',
 '유한회사올빼미하우스',
 '(주)미디어이야기',
 '(사)한국독립영화협회',
 '아이엠케이',
 '신씨네',
 '(주)문와쳐',
 'A라인',
 '영화사인연',
 '(주)베드픽쳐스',
 '(주)코코믹스',
 '(주)아이디어랩',
 '한맥영화',
 '무치필름',
 '(주)필름더데이즈',
 '뉴마켓필름',
 '제나두엔터테인먼트',
 '대우시네마',
 '(주)달빛공장',
 '필름에이픽쳐스',
 '투썸업픽쳐스',
 '씨드',
 '(주

In [283]:
for i in range(len(df)):
    if df.iloc[i]["배급사"] in movies:
        df["배급사"].replace(df.iloc[i]["배급사"], 0, inplace=True)

In [284]:
df['배급사'].replace("0", 0, inplace = True)

In [285]:
df['배급사'].value_counts()

0                          10888
(주)도키엔터테인먼트                  669
CJENM                        618
(유)조이앤컨텐츠그룹                  555
(주)영진크리에이티브                  403
롯데쇼핑㈜롯데엔터테인먼트                263
(주)소나무픽쳐스                    228
한국소니픽쳐스릴리징브에나비스타영화㈜          219
워너브러더스코리아(주)                 178
유니버설픽쳐스인터내셔널코리아(유)           177
(주)팝엔터테인먼트                   173
(주)쇼박스                       165
(주)영화사진진                     153
(주)넥스트엔터테인먼트월드(NEW)          144
이십세기폭스코리아(주)                 130
(주)영화사가을                     126
스크린조이                        122
(주)플라이어                      122
주식회사마운틴픽쳐스                   119
(주)스폰지이엔티                    112
와이드릴리즈(주)                    103
판씨네마(주)                       87
(주)디스테이션                      84
(주)미디어데이                      79
(주)박수엔터테인먼트                   76
주식회사루믹스미디어                    75
㈜인디스토리                        75
(주)영화사안다미로                    74
(주)엣나인필름                      73
(주)나우콘텐츠                      72
(주)영화사빅   

### 9. 감독

In [286]:
# 감독 1명만 남기기

for i in range(len(df["감독"])):
    if type(df["감독"][i]) == float:
        df["감독"][i] = 0
    else:
        df["감독"][i] = df["감독"][i].split(",")[0]

<ipython-input-286-7c328aa67c95>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감독"][i] = df["감독"][i].split(",")[0]


In [287]:
# 감독 공백 제거
for i in range(len(df["감독"])):
    if type(df["감독"][i]) == float:
        df["감독"][i] = 0
    else:
        List = df["감독"][i].split(" ")
        df["감독"][i] = "".join(List)

<ipython-input-287-9dac42f26a1c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감독"][i] = "".join(List)


In [288]:
df["감독"].replace(["없음", "개봉연도없음"], 0, inplace = True)

In [289]:
df["감독"].value_counts()

0          1905
강정훈          90
나가에타카미       75
스즈키켄이치로      72
김부곤          59
           ... 
후지이미치히토       1
톰나이젤          1
에칸보도간         1
블랙하트갱         1
조해동           1
Name: 감독, Length: 9456, dtype: int64

In [290]:
count = df["감독"].value_counts()

movies = []
for key, value in count.items():
    if value == 1:
        movies.append(key)

In [291]:
for i in range(len(df)):
    if df.iloc[i]["감독"] in movies:
        df["감독"].replace(df.iloc[i]["감독"], 0, inplace=True)

In [292]:
df["감독"].value_counts()

0          8525
강정훈          90
나가에타카미       75
스즈키켄이치로      72
김부곤          59
           ... 
하비에르페서        2
김양희           2
최경옥           2
토아프레이저        2
무라야마쿄스케       2
Name: 감독, Length: 2836, dtype: int64

### 10. 배우

In [293]:
# 배우 공백 제거
for i in range(len(df["배우"])):
    if type(df["배우"][i]) == float:
        df["배우"][i] = 0
    else:
        List = df["배우"][i].split(" ")
        df["배우"][i] = "".join(List)

<ipython-input-293-169ba4306bbc>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배우"][i] = "".join(List)


In [294]:
# 배우 3명만 남기기

for i in range(len(df["배우"])):
    if type(df["배우"][i]) == float:
        df["배우"][i] = 0
    else:
        df["배우"][i] = df["배우"][i].split(",")[0:3]

<ipython-input-294-a20a9924b785>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배우"][i] = df["배우"][i].split(",")[0:3]


In [295]:
# 1안 [배우1], [배우2], [배우3] 합-> [배우4]
#    0                                    6
#     박종혁 오승훈 최서현
#     10만     20만    30만  - 10만 => 3년 기간 내 평균 동원 관객수
#     a          b          c
#     a + b + c 

# 2안 [배우1, 배우2, 배우3] 평균-> [배우4]

   
    
df["배우"].head(100)

0                              [0]
1             [애덤존슨, 로버트제인, 케빈소르보]
2          [모건브래들리, 그레이엄빈센트, 캠벨루셀]
3                       [빌오버스트주니어]
4                              [0]
5       [안나알렌, 나초프레스네다, 메르세데스카스트로]
6                        [존W.바우어스]
7               [C.토마스하우웰, 티모시바톰즈]
8                 [타치바나케이코, 이치죠리노]
9                              [0]
10       [지노안소니페시, 브리나켈리, 제이슨스튜어트]
11        [버지니아가드너, 윌리엄매포더, 마크롤스톤]
12            [챈들러캔터베리, 라다미첼, 발킬머]
13          [에단호크, 안젤리나졸리, 키퍼서덜랜드]
14            [빈디젤, 사무엘L.잭슨, 루비로즈]
15                  [샤이아라보프, 사라로머]
16               [티모시올리펀트, 알렉스페티퍼]
17          [샤이아라보프, 미셸모나한, 도니제프콧]
18         [베네딕트컴버배치, 클레어포이, 숀에반스]
19                             [0]
20                        [캐서린아네트]
21            [존말코비치, 윌렘대포, 캐리엘위스]
22                  [아론에크하트, 벤킹슬리]
23        [사무엘L.잭슨, 케빈스페이시, 데이빗모스]
24                   [존트라볼타, 우마서먼]
25         [오셔잭슨주니어, 코리호킨스, 제이슨미첼]
26                 [제라드버틀러, 제이미폭스]
27           [크리스헴스워스, 테사톰슨, 리암니슨]
28            [빈디젤, 

In [296]:
df.to_csv("중간저장2.csv", encoding = 'utf-8-sig')

## preprocessing3

+ 매출액 누적매출액 음수 처리;

In [17]:
import pandas as pd

df = pd.read_csv("중간저장2.csv", index_col =0, thousands = ',' )
df

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,"416 프로젝트 ""공동의 기억: 트라우마""",NaN,2192500,2192500,437,437,3,17,한국,0,등급없음,다큐멘터리,0,['0']
1,미씨카: 더 네크로맨서,2017-11-02,300000,300000,60,60,1,1,미국,(주)제이에이와이이엔터테인먼트,15세이상관람가,판타지,0,"['애덤존슨', '로버트제인', '케빈소르보']"
2,월드 워 핵전쟁의 서막,2019-03-07,26000,26000,17,17,5,17,기타,주식회사풍경소리,15세이상관람가,액션,0,"['모건브래들리', '그레이엄빈센트', '캠벨루셀']"
3,울프라이징,2016-11-17,300000,300000,60,60,1,1,미국,주식회사루믹스미디어,15세이상관람가,공포,0,['빌오버스트주니어']
4,제3의앵글 : 형형색색대만전,NaN,1830000,1830000,305,305,1,2,기타,0,등급없음,장르없음,0,['0']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21320,댄싱 인 자파,NaN,88000,88000,15,15,1,2,미국,0,등급없음,다큐멘터리,힐라메달리아,['0']
21321,위기의 30대 여자들,NaN,946400,946400,136,136,1,2,기타,0,등급없음,다큐멘터리,힐라메달리아,['0']
21322,16세의 사운드트랙,2020-07-02,3543400,3543400,413,413,19,150,영국,(주)마노엔터테인먼트,12세이상관람가,코미디,0,"['제임스캘로웨이', '스칼렛마셜']"
21323,나의 말을 지켜봐: 어느 귀 먼 연예인들의 이야기,NaN,293500,293500,114,114,1,2,미국,0,등급없음,다큐멘터리,0,['0']


In [18]:
values = []

for value in df['매출액'].values:
    value = abs(value)
    values.append(value)

In [19]:
stacked_values = []

for value in df['누적매출액'].values:
    value = abs(value)
    stacked_values.append(value)

In [20]:
stacked_cus = []

for value in df['누적관객수'].values:
    value = abs(value)
    stacked_cus.append(value)

In [21]:
cus= []

for value in df['관객수'].values:
    value = abs(value)
    cus.append(value)

In [22]:
df['매출액'] = values
df['누적매출액'] = stacked_values
df['관객수'] = cus
df['누적관객수'] = stacked_cus

In [23]:
df.to_csv("중간저장3.csv", index = False, encoding = 'utf-8-sig', sep = ',')